## Description:
Extracts a csv file of the entire field from the imported nek output file.

In [ ]:
from __future__ import annotations
from pathlib import Path
import os
os.chdir("..")
import yaml
from flamekit.datasets import extract_full_field_csv
from flamekit.io_fronts import Case
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank


### Read config file and define helper functions

In [ ]:
PROJECT_ROOT = Path.cwd().resolve()
CONFIG_PATH = PROJECT_ROOT / "notebooks" / "configs" / "Extract_field.yaml"
CFG = yaml.safe_load(CONFIG_PATH.read_text(encoding="utf-8"))

DATA_BASE_DIR = PROJECT_ROOT / Path(CFG["DATA_BASE_DIR"])
OUTPUT_BASE_DIR = PROJECT_ROOT / Path(CFG["OUTPUT_BASE_DIR"])  
FILE_NAME = str(CFG["FILE_NAME"])

TIME_STEP = int(CFG["TIME_STEP"])
MULTI_TIME_STEP = bool(CFG["MULTI_TIME_STEP"])
TIME_STEP_START = int(CFG["TIME_STEP_START"])
TIME_STEP_END = int(CFG["TIME_STEP_END"])

PHI = float(CFG["PHI"])
LAT_SIZE = str(CFG["LAT_SIZE"])
T_REF = float(CFG["T_REF"])
P_REF = float(CFG["P_REF"])

POST = bool(CFG["POST"])
SCALARS = list(CFG["SCALARS"])

COMP_VEL_JACOBIAN = bool(CFG["COMP_VEL_JACOBIAN"])
COMP_VEL_HESSIAN = bool(CFG["COMP_VEL_HESSIAN"])
COMP_REACTION_RATES = bool(CFG["COMP_REACTION_RATES"])
COMP_T_GRAD = bool(CFG["COMP_T_GRAD"])
COMP_CURV_GRAD = bool(CFG["COMP_CURV_GRAD"])
COMP_LOCAL_VEL_JACOBIAN = bool(CFG["COMP_LOCAL_VEL_JACOBIAN"])

CANTERA_YAML = PROJECT_ROOT / Path(CFG["CANTERA_YAML"])
CANTERA_INPUTS = [str(CANTERA_YAML), None, T_REF, P_REF]

def _process_one_timestep(ts: int) -> None:
    case = Case(
        base_dir=OUTPUT_BASE_DIR,
        phi=PHI,
        lat_size=LAT_SIZE,
        time_step=ts,
        post=POST,
    )

    out_path = extract_full_field_csv(
        case,
        data_base_dir=DATA_BASE_DIR,
        output_base_dir=OUTPUT_BASE_DIR,
        file_name=FILE_NAME,
        scalar_names=SCALARS,
        comm=comm,
        compute_vel_jacobian=COMP_VEL_JACOBIAN,
        compute_vel_hessian=COMP_VEL_HESSIAN,
        compute_reaction_rates=COMP_REACTION_RATES,
        compute_T_grad=COMP_T_GRAD,
        compute_curv_grad=COMP_CURV_GRAD,
        compute_local_vel_jacobian=COMP_LOCAL_VEL_JACOBIAN,
        compute_progress_var=True,
        cantera_inputs=CANTERA_INPUTS,
    )

    if rank == 0:
        print(f"[rank0] Saved full field: {out_path}")



### Run

In [ ]:
if MULTI_TIME_STEP:
    for ts in range(TIME_STEP_START, TIME_STEP_END + 1):
        if rank == 0:
            print(f"=== Processing TIME_STEP={ts} ===")
        _process_one_timestep(ts)
else:
    if rank == 0:
        print(f"=== Processing TIME_STEP={TIME_STEP} ===")
    _process_one_timestep(TIME_STEP)

=== Processing TIME_STEP=270 ===
2026-01-07 21:17:06,458 - Mesh - INFO - Initializing empty Mesh object.
2026-01-07 21:17:06,458 - Field - INFO - Initializing empty Field object
2026-01-07 21:17:06,459 - pynekread - INFO - Reading file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/phi0.40/h400x025_ref/po_postPremix0.f00001
2026-01-07 21:17:06,469 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2026-01-07 21:17:06,470 - Mesh - INFO - Initializing common attributes.
2026-01-07 21:17:06,471 - Mesh - INFO - Getting vertices
2026-01-07 21:17:06,473 - Mesh - INFO - Getting edge centers
2026-01-07 21:17:06,513 - Mesh - INFO - Facet centers not available for 2D
2026-01-07 21:17:06,514 - Mesh - INFO - Mesh object initialized.
2026-01-07 21:17:06,514 - Mesh - INFO - Mesh data is of type: float64
2026-01-07 21:17:06,515 - Mesh - INFO - Elapsed time: 0.045407998s
2026-01-07 21:17:06,515 - pynekread - INFO - Reading field data
2026-01-07 21:17:06,515 - pynekrea

KeyboardInterrupt: 